### PYTORCH TO COREML
using the same basic windchill prediction model, training and trying to deploy in coreml
https://coremltools.readme.io/docs/pytorch-conversion

In [1]:
import torch
import os 
import numpy as np
import matplotlib.pyplot as plt

/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- load in weather data using only valid columns, leaving out the last column (wind chill label)
- create a tensor from all data

In [2]:
weather_data = np.genfromtxt("2022_data.csv",delimiter = ",", usecols = (0,1,2,3,4,6,8,9,11,12,15,17,18,19,22), skip_header = 1, filling_values=0.0)[:, :-1]
weather_tensor = torch.tensor(weather_data).float()
xdims = weather_tensor.shape[1] - 1


Make a pytorch DataSet from the tensor

In [3]:
weather_dataset = torch.utils.data.TensorDataset(weather_tensor[:, :-1], weather_tensor[:, -1:])


DataLoader 
puts data on the right device, shuffles, can use parallel programming, define batch size etc
- sees everything in an epoch
- next epoch, sees them in a different order
- suffles with 32 irows in a batch


In [4]:
loader = torch.utils.data.DataLoader(weather_dataset, shuffle=True, batch_size=32)


In [5]:
net = torch.nn.Sequential(
    torch.nn.Linear(xdims, 32), # takes in x dims columns, then funnels to a hidden layer of 32
    torch.nn.ELU(), 
    # ELU is an activation layer, like a sigmoidal function!
    torch.nn.Linear(32, 32),
    torch.nn.ELU(),
    torch.nn.Linear(32, 16),
    torch.nn.ELU(),
    torch.nn.Linear(16, 1)
)


In [6]:
# Mean squared error loss function
criterion = torch.nn.MSELoss()


In [7]:
# optimizer called Adam, using a learning rate of 1e-4
opt = torch.optim.Adam(net.parameters(), 1e-4 )


Training loop

In [8]:
for ep in range(1000):
    # every epoch sees every row exactly one time, but inputs are shuffled every epoch
    total_loss = 0.0
    
    # batch by batch
    for batch in loader:
        
        opt.zero_grad()
        
        batchX = batch[0]
        batchY = batch[1]

        pred = net(batchX)
        loss = criterion(pred, batchY)
        
        loss.backward()
        opt.step()
        total_loss += loss



In [9]:
#pip3 install coremltools==5.0b5 protobuf==3.20.1



In [10]:
## This code will print out the real, then expected for every batch of 32 
# for batch in loader:
#     batchX = batch[0]
#     batchY = batch[1]
#     print(batchY)
#     print(net(batchX))
    

### Quantize and convert to TorchScript
The process of tracing takes an example input and traces its flow through the model. You can trace the model by creating an example image input, as shown in the above code using random data. To understand the reasons for tracing and how to trace a PyTorch model, see Model Tracing.


If your model uses a data-dependent control flow, such as a loop or conditional, the traced model won't generalize to other inputs. In such cases you can experiment with applying PyTorch's JIT script (torch.jit.script) to your model as described in Model Scripting. You can also use a combination of tracing and scripting.



In [11]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    net, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)
# set model to evaluation mode
model_dynamic_quantized.eval()
example_tensor = weather_tensor[0,:-1]
# convert to torch script
traced_script_module = torch.jit.trace(net, example_tensor)


In [12]:
out = traced_script_module(example_tensor)

### Convert to CoreML Model

In [13]:
import coremltools as ct
traced_script_module.eval()
# Using image_input in the inputs parameter:
# Convert to Core ML program using the Unified Conversion API.
model = ct.convert(
    traced_script_module,
    inputs=[ct.TensorType(shape=example_tensor.shape)]
 )

# Save the converted model.
model.save("newmodel.mlmodel")

Converting PyTorch Frontend ==> MIL Ops:  92%|▉| 12/13 [00:00<
Running MIL Common passes:   0%|  | 0/40 [00:00<?, ? passes/s]/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:107: UserWarning: Input, 'input.1', of the source model, has been renamed to 'input_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:135: UserWarning: Output, '26', of the source model, has been renamed to 'var_26' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL Common passes: 100%|█| 40/40 [00:00<00:00, 3186.80
Running MIL Clean up passes: 100%|█| 11/11 [00:00<00:00, 3556.
Translating MIL ==> NeuralNetwork Ops: 100%|█| 18/18 [00:00<00


### Convert to CoreML Package

In [14]:
import coremltools as ct
traced_script_module.eval()
# Using image_input in the inputs parameter:
# Convert to Core ML program using the Unified Conversion API.
model = ct.convert(
    traced_script_module,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_tensor.shape)]
 )

Converting PyTorch Frontend ==> MIL Ops:  92%|▉| 12/13 [00:00<
Running MIL Common passes: 100%|█| 40/40 [00:00<00:00, 5406.08
Running MIL FP16ComputePrecision pass: 100%|█| 1/1 [00:00<00:0
Running MIL Clean up passes: 100%|█| 11/11 [00:00<00:00, 1362.


got this message: 

Model is not in eval mode. Consider calling '.eval()' on your model prior to conversion


ALSO: 
As an alternative, you can convert the model to a neural network by eliminating the convert_to parameter:

In [35]:
# Save the converted model.
model.save("newmodel.mlpackage")

### success! (maybe?)
next step is to try integrating it into my app:
https://developer.apple.com/documentation/coreml/integrating_a_core_ml_model_into_your_app

The convert to ml code yields this attribute error: 

AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
    
    
trying to downgrade numpy: 
conda install -c conda-forge numpy=1.19.5 
(a 2020 version)

--> didnʻt work in my terminal. 

trying uninstall numpy
--> worked

conda install numpy=1.19.5  

--> doesnt work because my python version is too high 

Specifications:

  - numpy=1.19.5 -> python[version='3.6.*|3.7.*|3.9.*|3.8.*']
  - numpy=1.19.5 -> python[version='>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.8,<3.9.0a0|>=3.9,<3.10.0a0']

Your python: python=3.11 (need python 3.6-3.9)

if this doesnt work, will try editing sourcecode locatlly to correct for the attribute error
--> looks like that exists in too many places. 

next, I will try downgrading my python version. First, must fiugre out the minimum python requirement for other pytorch operations

conda install python=3.9

then 

conda install numpy=1.19.5  

then will retry


--> now it doesnʻt recognize import torch, so reinstalling pytorch

 conda install pytorch torchvision torchaudio -c pytorch

### Optimize for pytorch model (old code)

In [36]:
# from torch.utils.mobile_optimizer import optimize_for_mobile
# # optimize for mobile so that we can export and use in Swift app
# torchscript_model_optimized = optimize_for_mobile(traced_script_module)

# # save as .pt 
# path = os.path.join(os.getcwd(),"model.pt")
# torchscript_model_optimized._save_for_lite_interpreter(path)


### Test

In [22]:
test = weather_tensor[0,:-1]
label = weather_tensor[0,-1]
label

tensor(4.9000)

In [23]:
print(net(test))

# net([2.4008e+02, 1.2300e+01, 2.0900e+01, 4.5000e+00, 1.5500e+01, 9.0000e-01,
#         8.4400e+01, 6.0000e-01, 1.0000e-01, 2.3000e+01, 0.0000e+00, 1.0143e+03,
#         0.0000e+00])
print(test)

tensor([9.4963], grad_fn=<AddBackward0>)
tensor([2.4008e+02, 1.2300e+01, 2.0900e+01, 4.5000e+00, 1.5500e+01, 9.0000e-01,
        8.4400e+01, 6.0000e-01, 1.0000e-01, 2.3000e+01, 0.0000e+00, 1.0143e+03,
        0.0000e+00])


NOTES FROM CORY: HOW TO SPLIT UP training, validation, testing

def train(loader):
    one tound of training with all data in the loader
    
    
def eval(loader):
    feed evyerhting in batches to model, adds up and takes the losses of the batches
    

